In [1]:
import pandas as pd

In [2]:
OrderBookPlots2002_df = pd.read_csv('./resources/2002.csv')
OrderBookPlots2002_df = OrderBookPlots2002_df[OrderBookPlots2002_df.DATE.isin(['2002-08-12', '2002-08-13'])]
# OrderBookPlots2002_df

In [3]:
OrderBookPlots2002_df[(OrderBookPlots2002_df.PRICE != OrderBookPlots2002_df.TRADEPRICE) &
                 pd.notnull(OrderBookPlots2002_df.TRADEPRICE)]

,Unnamed: 0,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATE,OBPLOTNO
1356,1356,1357,RU0009029540,S,103002000,1354,2,0.000,5,29844786.0,4420.000,2002-08-12,48
1382,1382,1383,RU0008926621,B,103010000,1374,2,2.900,2100,29844787.0,2.810,2002-08-12,530
1416,1416,1417,RU0008926621,S,103015000,1397,2,2.800,1200,29844788.0,2.801,2002-08-12,167
1480,1480,1481,RU14MSNG3008,B,103023000,1439,2,0.800,68500,29844794.0,0.799,2002-08-12,96
1483,1483,1484,RU0008926621,S,103023000,1441,2,2.801,30000,29844795.0,2.810,2002-08-12,530
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185377,94633,94634,RU0008926621,B,184458000,41551,2,3.600,30000,29878363.0,2.894,2002-08-13,42428
185379,94635,94636,RU0008926621,B,184458000,41551,2,3.600,100000,29878364.0,2.894,2002-08-13,42428
185381,94637,94638,RU0008926621,B,184458000,41551,2,3.600,68300,29878365.0,2.895,2002-08-13,42428
185383,94639,94640,RU0008926621,B,184458000,41551,2,3.600,26900,29878366.0,2.895,2002-08-13,42428


In [35]:
OrderBookPlots2007_df = pd.read_csv('./resources/LKOH_SBER_GMKN_2007_10.csv')
# OrderBookPlots2007_df = pd.read_csv('./resources/LKOH_2007_10.csv')
OrderBookPlots2007_df = OrderBookPlots2007_df[OrderBookPlots2007_df.DATE.isin(['2007-10-01', '2007-10-02'])]
# OrderBookPlots2007_df

In [5]:
OrderBookPlots2007_df[(OrderBookPlots2007_df.PRICE != OrderBookPlots2007_df.TRADEPRICE) &
                 pd.notnull(OrderBookPlots2007_df.TRADEPRICE)]

,Unnamed: 0,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATE,OBPLOTNO
2336,15529,15530,LKOH,S,103001000,15442,2,0.00,400,329668177.0,2066.11,2007-10-01,122
2359,15672,15673,LKOH,B,103002000,15522,2,2069.00,2,329668201.0,2067.00,2007-10-01,541
2361,15676,15677,LKOH,B,103002000,15524,2,0.00,18,329668203.0,2067.00,2007-10-01,541
2363,15680,15681,LKOH,S,103002000,15528,2,2060.00,55,329668205.0,2066.11,2007-10-01,122
2365,15682,15683,LKOH,S,103002000,15581,2,2065.90,100,329668206.0,2066.11,2007-10-01,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...
521656,1132248,1132249,GMKN,S,174457000,476469,2,6644.00,5,330414209.0,6644.10,2007-10-02,102950
521658,1132250,1132251,GMKN,S,174457000,476469,2,6644.00,1,330414210.0,6644.07,2007-10-02,102950
521678,1132345,1132346,SBER03,B,174458000,476517,2,104.94,200,330414238.0,104.89,2007-10-02,49790
521681,1132350,1132351,LKOH,B,174458000,476519,2,2112.01,5,330414240.0,2112.00,2007-10-02,63756


In [40]:
class OBPlots2OBPlots_Attributes:
    def __init__(self, OBPs_df):
        self.OBPs_df = OBPs_df
        self.secs = pd.unique(self.OBPs_df.SECCODE)
        self.dates = pd.unique(self.OBPs_df.DATE)
        self.obplotnos = pd.unique(self.OBPs_df.OBPLOTNO)
        self.obp_atts = {
            sec:datedic for (sec, datedic) in zip(
                self.secs,
                [{
                    date:obps_attsdic for (date, obps_attsdic) in zip(
                        self.dates, 
                        [{} for i in range(len(self.dates))]
                    )
                } for i in range(len(self.secs))]
            )
        }
        self.iterate_ob_rows()
    
    # Accordin to specs ACTION 0 shows balance of order, not cancelled volune of the order, IT'S WRONG
    # Actually, it shows cancelled volune of the order
    def cur_CBOVOL(self, LAST_CBOVOL, BUYSELL, ACTION, PRICE, VOLUME):
        if (BUYSELL == 'B') & (ACTION == 0):
            return(LAST_CBOVOL + (PRICE * VOLUME))
        else:
            return(LAST_CBOVOL)
        
    def cur_CSOVOL(self, LAST_CSOVOL, BUYSELL, ACTION, PRICE, VOLUMEN):
        if (BUYSELL == 'S') & (ACTION == 0):
            return(LAST_CSOVOL + (PRICE * VOLUME))
        else:
            return(LAST_CSOVOL)
        
    def cur_BOVOL(self, LAST_BOVOL, BUYSELL, ACTION, PRICE, VOLUME):
        if (BUYSELL == 'B') & (ACTION == 1):
            return(LAST_BOVOL + (PRICE * VOLUME))
        else:
            return(LAST_BOVOL)
        
    def cur_SOVOL(self, LAST_SOVOL, BUYSELL, ACTION, PRICE, VOLUME):
        if (BUYSELL == 'S') & (ACTION == 1):
            return(LAST_SOVOL + (PRICE * VOLUME))
        else:
            return(LAST_SOVOL)
        
    def ob_row2obp_atts(self, SECCODE, DATE, NO, BUYSELL, TIME, ORDERNO, 
                        ACTION, PRICE, VOLUME, TRADEPRICE, OBPLOTNO):
        print(SECCODE)
        print(self.obp_atts[SECCODE])
        if OBPLOTNO not in self.obp_atts[SECCODE][DATE]:
            self.obp_atts[SECCODE][DATE][OBPLOTNO] = {}
            
        cur_obps_attsdic_len = len(self.obp_atts[SECCODE][DATE][OBPLOTNO])
        if cur_obps_attsdic_len > 0:
            last_key = list(self.obp_atts[SECCODE][DATE][OBPLOTNO])[-1]
            LAST_CBOVOL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key]['CBOVOL']
            LAST_CSOVOL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key]['CSOVOL']
            LAST_BOVOL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key]['BOVOL']
            LAST_SOVOL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key]['SOVOL']
            LAST_BTVOL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key]['BTVOL']
            LAST_STVOL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key]['STVOL']
#             LAST_TBNET = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key][TBNET]
#             LAST_TSNET = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key][TSNET]
#             LAST_OBP_BAL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key][OBP_BAL]
#             LAST_DAY_BAL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key][DAY_BAL]
        else:
            LAST_CBOVOL = 0
            LAST_CSOVOL = 0
            LAST_BOVOL = 0
            LAST_SOVOL = 0
            LAST_BTVOL = 0
            LAST_STVOL = 0
            LAST_TBNET = 0
            LAST_TSNET = 0
            LAST_OBP_BAL = 0
            LAST_DAY_BAL = 0
        self.obp_atts[SECCODE][DATE][OBPLOTNO][NO] = {
            'CBOVOL': self.cur_CBOVOL(LAST_CBOVOL, BUYSELL, ACTION, PRICE, VOLUME), # cancel buy order volume
            'CSOVOL': self.cur_CSOVOL(LAST_CSOVOL, BUYSELL, ACTION, PRICE, VOLUME), # cancel sell order volume
            'BOVOL': self.cur_BOVOL(LAST_BOVOL, BUYSELL, ACTION, PRICE, VOLUME), # buy order volume
            'SOVOL': self.cur_SOVOL(LAST_SOVOL, BUYSELL, ACTION, PRICE, VOLUME)}#, # sell order volume
#             'BTVOL': self.cur_BTVOL(), # buy trade volumne
#             'STVOL': self.cur_STVOL(), # sell trade volume
#             'TBNET': self.cur_TBNET(), # trade volumne - buy volume <= 0
#             'TSNET': self.cur_TSNET(), # trade volume - sell volume >= 0
#             'OBP_BAL': self.cur_OBP_BAL(), # oreder-book-plot balance
#             'DAY_BAL': self.cur_DAY_BAL() } # trading day balance
        
    def iterate_ob_rows(self):
        [self.ob_row2obp_atts(SECCODE, DATE, NO, BUYSELL, TIME, ORDERNO, 
                              ACTION, PRICE, VOLUME, TRADEPRICE, OBPLOTNO)
        for (SECCODE, DATE, NO, BUYSELL, TIME, ORDERNO, 
             ACTION, PRICE, VOLUME, TRADEPRICE, OBPLOTNO)
         in zip(self.OBPs_df.SECCODE,
                self.OBPs_df.DATE,
                self.OBPs_df.NO,
                self.OBPs_df.BUYSELL,
                self.OBPs_df.TIME,
                self.OBPs_df.ORDERNO,
                self.OBPs_df.ACTION,
                self.OBPs_df.PRICE,
                self.OBPs_df.VOLUME,
                self.OBPs_df.TRADEPRICE,
                self.OBPs_df.OBPLOTNO)]

In [41]:
obp_atts = OBPlots2OBPlots_Attributes(OrderBookPlots2002_df)

RU14VOST1005
{'2002-08-12': {}, '2002-08-13': {}}
RU0009071187
{'2002-08-12': {}, '2002-08-13': {}}
RU14VOST1005
{'2002-08-12': {0: {1: {'CBOVOL': 0, 'CSOVOL': 0, 'BOVOL': 0, 'SOVOL': 10710.0}}}, '2002-08-13': {}}
RU0009100762
{'2002-08-12': {}, '2002-08-13': {}}
RU0009054449
{'2002-08-12': {}, '2002-08-13': {}}
RU14VOST1005
{'2002-08-12': {0: {1: {'CBOVOL': 0, 'CSOVOL': 0, 'BOVOL': 0, 'SOVOL': 10710.0}}, 2: {3: {'CBOVOL': 0, 'CSOVOL': 0, 'BOVOL': 0, 'SOVOL': 52272.0}}}, '2002-08-13': {}}
RU0009029532
{'2002-08-12': {}, '2002-08-13': {}}
RU0008926258
{'2002-08-12': {}, '2002-08-13': {}}
RU0009029532
{'2002-08-12': {0: {7: {'CBOVOL': 0, 'CSOVOL': 0, 'BOVOL': 22750.0, 'SOVOL': 0}}}, '2002-08-13': {}}
RU14MSNG3008
{'2002-08-12': {}, '2002-08-13': {}}
RU14TATN3006
{'2002-08-12': {}, '2002-08-13': {}}
RU14MSNG3008
{'2002-08-12': {0: {10: {'CBOVOL': 0, 'CSOVOL': 0, 'BOVOL': 0, 'SOVOL': 8700.0}}}, '2002-08-13': {}}
RU14MSNG3008
{'2002-08-12': {0: {10: {'CBOVOL': 0, 'CSOVOL': 0, 'BOVOL': 0, 'S

KeyError: 'BTVOL'

In [ ]:
obp_atts.obp_atts.keys()

In [ ]:
obp_atts.obp_atts['RU0009024277'].keys()

In [ ]:
obp_atts.obp_atts['RU0009024277']['2002-08-12'].keys()

In [ ]:
obp_atts.obp_atts['RU0009024277']['2002-08-12'][30].keys()

In [ ]:
tmp = {1:{2:5, 6:7} , 3:4}
# tmp = {}
# list(tmp)[-1]
len(tmp)

In [ ]:
tmp[1][6]

In [ ]:
{'c': 2}

In [ ]:
len(tmp[1])